In [7]:
import h5py
import numpy as np
import tensorflow as tf
import pathlib
import pickle
from sklearn.preprocessing import OneHotEncoder
import sys
from tqdm import tqdm

In [8]:


import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
attack_vector='cache' #network
attack_vector='network'
machine_setting='dynamic_browser_version' # 'static_browser_version'
machine_setting= 'static_browser_version'
n_dataset_features = 15000 if attack_vector == 'cache' else 3000
root_path=f'/content/drive/MyDrive/Colab Notebooks/concept-drift-online'
data_path=f'{root_path}/data/{machine_setting}/{attack_vector}'
model_0_train_path=f'2021_08_27/train.h5'


In [14]:
sys.path.append(str(root_path)) # Add the directory to the path
from utils import cnn_lstm,prepare_hdf5_data,OpenWorldLabelEncoder,small_cnn_lstm

In [16]:
# Example usage:
hdf5_train_path = (pathlib.Path(data_path)/model_0_train_path).as_posix()

nn_config_cache={
    'features_per_layer': 256,
    'strides': 3,
    'pool_size': 4,
    'units': 32,
    'dropout': 0.2,
}

nn_config_network={
            'features_per_layer': 256,
            'pool_size': 3,
            'strides': 3,
            'units': 128,
            'dropout': 0.8,
            'kernel_size': 32,
}

network_config=nn_config_cache if attack_vector == 'cache' else nn_config_network
model_func=cnn_lstm if attack_vector == 'cache' else small_cnn_lstm

new_model=model_func(n_dataset_features=n_dataset_features, num_of_classes=100, **network_config)
le=OneHotEncoder()

with h5py.File(hdf5_train_path, 'r') as hdf:
    le.fit( sorted(hdf['site_name'][:].reshape(-1,1)) )

le_dir=f'{data_path}/models/label_encoder_100.pkl'
with open(le_dir, 'wb') as f:
    pickle.dump(le, f)


In [17]:
batch=256
max_epochs=100
datasets=prepare_hdf5_data(hdf5_train_path,le,batch_size=batch,n_samples=-1,sample_validation=0.1,verbose=0)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

verbose=1
new_model.fit(datasets['data'],
            validation_data=datasets['val'] ,
            epochs=max_epochs,
            callbacks=[early_stopping],
            verbose=verbose
        )


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 151s 4s/step - accuracy: 0.3612 - loss: 2.7467 - val_accuracy: 0.8709 - val_loss: 0.4641
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.8968 - loss: 0.4024 - val_accuracy: 0.9057 - val_loss: 0.3308
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 136s 4s/step - accuracy: 0.9336 - loss: 0.2373 - val_accuracy: 0.9259 - val_loss: 0.2865
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.9521 - loss: 0.1813 - val_accuracy: 0.9282 - val_loss: 0.2558
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.9516 - loss: 0.1735 - val_accuracy: 0.9338 - val_loss: 0.2674
Epoch 6/100
20/32 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.9533 - loss: 0.1510

In [ ]:
model_path=f'{data_path}/models/cnn_lstm.keras'
new_model.save(model_path)

In [ ]:
new_model.summary()

In [ ]:
n_open_world_labels=30
num_of_classes=100-n_open_world_labels
owle=OpenWorldLabelEncoder(le,num_of_classes)
datasets_ow=prepare_hdf5_data(hdf5_train_path,owle,batch_size=batch,n_samples=-1,sample_validation=0.1,verbose=0)
ow_model=cnn_lstm(n_dataset_features=15000, num_of_classes=num_of_classes, **network_config)
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

verbose=1
ow_model.fit(datasets_ow['data'],
            validation_data=datasets_ow['val'] ,
            epochs=max_epochs,
            callbacks=[early_stopping],
            verbose=verbose
        )

In [ ]:
ow_model_path=f'{data_path}/models/cnn_lstm_ow.keras'
ow_model.save(ow_model_path)

In [ ]:
ow_model.summary()